In [1]:
# Description:
# This script processes 3D models from a specified input folder. For each model
# file (with a .off extension), it generates 12 uniformly-rotated-view images
# and saves them to a corresponding sub-directory in the output folder.
#
# The script performs the following steps for each model:
#   1. Loads the .off mesh file.
#   2. Normalizes the mesh size and centers it.
#   3. Sets up a visualizer with a white background and lighting.
#   4. Rotates the object in 12 steps around the Z-axis.
#   5. Captures a screen image at each step.
#   6. Saves the images to a dedicated folder named after the object.
#
# This version is modified to handle batch processing of all files in a folder.
#

# ==============================================================================
# 1. SETUP AND INSTALLATION
# ==============================================================================


import importlib.util
import sys
import subprocess
import os
import open3d as o3d
import numpy as np
import time

# Check if open3d is installed and install it if necessary
package_name = 'open3d'
spec = importlib.util.find_spec(package_name)

if spec is None:
    print(f"📦 Installing {package_name}...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        # Re-import after installation
        import open3d as o3d
        print(f"✅ {package_name} installed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package_name}. Please install it manually using 'pip install open3d'. Error: {e}")
        sys.exit(1)
else:
    print(f"✅ {package_name} is already installed.")


# ==============================================================================
# 2. CONFIGURATION & MAIN PROCESSING
# ==============================================================================

# --- USER-CONFIGURABLE PATHS ---
# 📁 Set the path to your folder containing .off files
input_dir = "/Users/sdas1/Documents/ModelNet10/bathtub/try/"  # <-- Provide Input Directory Path here!
# 📁 Set the path where the output images will be saved
output_dir = "/Users/sdas1/Documents/views_hold/bathtub/try/"  # <-- Provide Ouput Directory Path here!
# --- END OF CONFIGURATION ---

# Ensure the main output directory exists
os.makedirs(output_dir, exist_ok=True)
print(f"✅ Output will be saved to: {output_dir}")

# Get a list of all files to process
try:
    all_files = os.listdir(input_dir)
    off_files = [f for f in all_files if f.endswith(".off")]
    if not off_files:
        print(f"⚠️ No .off files found in the directory: {input_dir}")
        sys.exit(0)
    print(f"🔍 Found {len(off_files)} '.off' files to process.")
except FileNotFoundError:
    print(f"❌ Input directory not found at: {input_dir}")
    sys.exit(1)


# Create ONE visualizer instance and reuse it for all models
print("🎨 Setting up visualization window...")
vis = o3d.visualization.Visualizer()
vis.create_window(visible=True) # Set to False if you don't want to see the window

# Main loop to process each .off file
for filename in off_files:
    file_path = os.path.join(input_dir, filename)
    object_id = os.path.splitext(os.path.basename(filename))[0]
    print(f"\n{'='*60}\n🔄 Processing model: {object_id}\n{'='*60}")

    # Create a dedicated output directory for the current object's views
    object_output_dir = os.path.join(output_dir, object_id)
    os.makedirs(object_output_dir, exist_ok=True)

    # Load the .off file
    print("  📂 Loading mesh from file...")
    mesh = o3d.io.read_triangle_mesh(file_path)

    # Check if mesh loaded successfully
    if len(mesh.vertices) == 0:
        print(f"  ❌ Failed to load mesh or mesh is empty for {filename}. Skipping.")
        continue # Skip to the next file
    
    print(f"  ✅ Mesh loaded successfully with {len(mesh.vertices)} vertices and {len(mesh.triangles)} triangles.")

    # --- Mesh Preparation ---
    # Create a fresh copy to avoid carrying over rotations
    mesh_copy = o3d.geometry.TriangleMesh(mesh)
    
    # Center the mesh and move it slightly above the ground plane
    mesh_copy.translate(-mesh_copy.get_center() + np.array([0, 0, 0.1]))
    
    # Normalize the mesh to a unit cube for consistent sizing
    max_bound = np.max(mesh_copy.get_max_bound() - mesh_copy.get_min_bound())
    mesh_copy.scale(1.0 / max_bound, center=mesh_copy.get_center())
    
    # Add colors if the mesh doesn't have them
    if not mesh_copy.has_vertex_colors():
        mesh_copy.paint_uniform_color([0.7, 0.7, 0.7])  # Light gray
        
    # Compute normals for proper lighting and visualization
    mesh_copy.compute_vertex_normals()

    print("  📸 Generating multi-view images...")

    # --- Visualization Setup for Current Model ---
    vis.clear_geometries()
    vis.add_geometry(mesh_copy)

    # Set render options
    render_option = vis.get_render_option()
    render_option.background_color = np.asarray([1.0, 1.0, 1.0]) # White background
    render_option.mesh_show_back_face = True
    render_option.light_on = True

    # Set camera position
    view_control = vis.get_view_control()
    view_control.set_zoom(0.7)
    view_control.set_front([1, 0, 0])  # Initial front direction
    view_control.set_up([0, 0, 1])     # Up direction
    view_control.set_lookat([0, 0, 0]) # Look at the center of the object

    # --- View Generation Loop ---
    n_views = 12
    rotation_angle = 360.0 / n_views  # degrees per step

    for i in range(n_views):
        # Create rotation matrix around the Z-axis (up-axis)
        rotation_matrix = mesh_copy.get_rotation_matrix_from_xyz((0, 0, np.radians(i * rotation_angle)))
        # Apply the rotation
        mesh_copy.rotate(rotation_matrix, center=(0, 0, 0))
        
        # Update the geometry in the visualizer to reflect the rotation
        vis.update_geometry(mesh_copy)
        vis.poll_events()
        vis.update_renderer()
        
        # Small delay to ensure rendering is complete before capture
        time.sleep(0.1)
        
        # Capture the current view
        output_filename = os.path.join(object_output_dir, f"{object_id}_view_{i:02d}.png")
        vis.capture_screen_image(output_filename, do_render=True)
    
    print(f"  ✅ Saved {n_views} views to: {object_output_dir}")

# Clean up and close the visualizer window
vis.destroy_window()

print(f"\n{'='*60}\n🎉 All models processed successfully!\n{'='*60}")


✅ open3d is already installed.
✅ Output will be saved to: /Users/sdas1/Documents/views_hold/bathtub/try/
🔍 Found 2 '.off' files to process.
🎨 Setting up visualization window...

🔄 Processing model: bathtub_0107
  📂 Loading mesh from file...
  ✅ Mesh loaded successfully with 1568 vertices and 1820 triangles.
  📸 Generating multi-view images...
  ✅ Saved 12 views to: /Users/sdas1/Documents/views_hold/bathtub/try/bathtub_0107

🔄 Processing model: bathtub_0108
  📂 Loading mesh from file...
  ✅ Mesh loaded successfully with 352 vertices and 322 triangles.
  📸 Generating multi-view images...
  ✅ Saved 12 views to: /Users/sdas1/Documents/views_hold/bathtub/try/bathtub_0108

🎉 All models processed successfully!
